## Group Level Analysis

This notebook is designed to perform a voxelwise analysis of the infant CBF data

In [1]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.fsl.utils import Merge
from nipype.interfaces.fsl.model import Randomise, Cluster
from nipype.interfaces.fsl.maths import ApplyMask

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#other study-specific variables
#project_home = '/Volumes/iang/active/BABIES/BABIES_perfusion'
project_home = '/Users/catcamacho/Box/SNAP/BABIES'
output_dir = project_home + '/proc/asl_group_affect'
preproc_dir = project_home + '/proc/asl_preproc'
wkflow_dir = project_home + '/workflows'
template = project_home + '/templates/T2w_BABIES_template_2mm.nii.gz'
mask = project_home + '/templates/BABIES_gm_mask_2mm.nii.gz'

# Files for group level analysis
group_mat = project_home + '/misc/design_affect.mat'
t_contrasts = project_home + '/misc/tcon_affect.con'

subjects_list = open(project_home + '/misc/subjects_asl_temp.txt').read().splitlines()
print(subjects_list)

['002x', '012', '020', '021', '023', '027', '031', '033x', '035', '039', '064x', '067', '072', '076x', '077x', '087', '091', '099', '100', '102', '105', '106', '109', '110', '115', '123', '125', '132', '140', '1005', '1010', '1020', '1023', '1024', '1027', '1033', '1036', '1078']


In [2]:
# Data Handling Nodes

datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

grabcbfdata = Node(DataGrabber(template=preproc_dir + '/cbf_volume/*/salsop_cbf_masked.nii.gz', 
                               sort_filelist=True, 
                               outfields=['cbf_list']), 
                   name='grabcbf')


In [3]:
def get_cluster_peaks(clusters_file, stat_file):
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, unravel_index, max
    from os.path import abspath
    
    # load up clusters
    clusters_nii = load(clusters_file)
    clusters_data = clusters_nii.get_data()
    cluster_labels, cluster_sizes = unique(clusters_data, return_counts=True)
    cluster_sizes = cluster_sizes[cluster_labels>0]
    cluster_labels = cluster_labels[cluster_labels>0]
    
    # set up dataframe
    cluster_info = DataFrame(columns=['clust_num','peak','num_voxels','X','Y','Z'])
    cluster_info['clust_num'] = Series(cluster_labels,index=None)
    
    for i in range(0,len(cluster_labels)):
        # load up stat image
        stat_nii = load(stat_file)
        stat_data = stat_nii.get_data()
        stat_data[clusters_data!=cluster_labels[i]]=0
        location=unravel_index(stat_data.argmax(), stat_data.shape)
        cluster_info.iloc[i,0]=cluster_labels[i]
        cluster_info.iloc[i,1]=max(stat_data)
        cluster_info.iloc[i,2]=cluster_sizes[i]
        cluster_info.iloc[i,3]=location[0]
        cluster_info.iloc[i,4]=location[1]
        cluster_info.iloc[i,5]=location[2]
    
    out_prefix = clusters_file[:-7]
    cluster_info.to_csv(out_prefix + '_peaks.csv')
    cluster_info_file = abspath(out_prefix + '_peaks.csv')
    return(cluster_info_file)

def extract_cluster_betas(cluster_index_file, sample_betas, min_clust_size, subject_ids):
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, zeros_like, invert
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath, basename
    
    sample_data = DataFrame(subject_ids, index=None, columns=['Subject'])
    
    cluster_nifti = load(cluster_index_file)
    cluster_data = cluster_nifti.get_data()
    clusters, cluster_sizes = unique(cluster_data, return_counts=True)
    cluster_sizes = cluster_sizes[clusters>0]
    clusters = clusters[clusters>0]
    clusters = clusters[cluster_sizes>min_clust_size]
    cluster_sizes = cluster_sizes[cluster_sizes>min_clust_size]
    ind_filename = basename(cluster_index_file) 
    out_prefix = ind_filename[:-7]
    
    for clust_idx in clusters:
        temp = zeros_like(cluster_data)
        temp[cluster_data==clust_idx] = 1
        temp_nii = Nifti1Image(temp,cluster_nifti.affine)
        temp_file = 'temp_clust_mask.nii.gz'
        save(temp_nii, temp_file)

        eb = ImageMeants()
        eb.inputs.in_file = sample_betas
        eb.inputs.mask = temp_file
        eb.inputs.out_file = 'betas.txt'
        eb.run()
        betas = open('betas.txt').read().splitlines()
        sample_data['clust' + str(clust_idx)] = Series(betas, index=sample_data.index)
    
    sample_data.to_csv(out_prefix+'_extracted_betas.csv')
    extracted_betas_csv = abspath(out_prefix+'_extracted_betas.csv')

    return(extracted_betas_csv)

In [4]:
# Analysis Nodes

merge = Node(Merge(dimension = 't'), name = 'merge')

apply_mask = Node(ApplyMask(mask_file=mask, nan2zeros=True), name='apply_mask')

randomise = Node(Randomise(tfce = True,
                           num_perm = 1000,
                           tcon = t_contrasts,
                           demean = True,
                           design_mat = group_mat), name = 'randomise')

cluster = MapNode(Cluster(out_localmax_txt_file = 'cluster_stats.txt',
                          threshold=2.48, 
                          out_index_file='clusters.nii.gz'), 
                  name='cluster', iterfield=['in_file'])

get_peaks = MapNode(Function(input_names=['clusters_file', 'stat_file'], 
                             output_names=['cluster_info_file'], 
                             function=get_cluster_peaks), 
                    name='get_peaks', iterfield=['clusters_file', 'stat_file'])

get_betas = MapNode(Function(input_names=['cluster_index_file', 'sample_betas', 
                                          'min_clust_size', 'subject_ids'], 
                             output_names=['extracted_betas_csv'], 
                             function=extract_cluster_betas), 
                    name='get_betas', iterfield=['cluster_index_file'])
get_betas.inputs.subject_ids = subjects_list
get_betas.inputs.min_clust_size=10

In [5]:
# Analysis workflow

grouplevel = Workflow(name='grouplevel_affect')

grouplevel.connect([(grabcbfdata, merge,[('cbf_list', 'in_files')]),
                    (merge, apply_mask, [('merged_file','in_file')]),
                    (apply_mask, randomise, [('out_file', 'in_file')]),
                    (randomise, cluster, [('tstat_files','in_file')]),
                    (cluster, get_peaks, [('index_file','clusters_file')]),
                    (randomise, get_peaks, [('tstat_files','stat_file')]),
                    (cluster, get_betas ,[('index_file','cluster_index_file')]),
                    (merge, get_betas, [('merged_file','sample_betas')]),
                    
                    (get_peaks, datasink, [('cluster_info_file','cluster_stats')]),
                    (cluster, datasink, [('index_file','cluster_file')]),
                    (get_betas, datasink, [('extracted_betas_csv','cluster_betas')]),
                    (randomise, datasink, [('t_corrected_p_files', 'factors_t_corrected_p_files')]),
                    (randomise, datasink, [('tstat_files', 'factors_tstat_files')])
                   ])

grouplevel.base_dir = wkflow_dir
grouplevel.write_graph(graph2use='flat')
grouplevel.run('MultiProc', plugin_args={'n_procs': 2})


181004-14:07:10,928 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/workflows/grouplevel_affect/graph.png (graph2use=flat, simple_form=True).
181004-14:07:10,970 workflow INFO:
	 Workflow grouplevel_affect settings: ['check', 'execution', 'logging', 'monitoring']
181004-14:07:10,987 workflow INFO:
	 Running in parallel.
181004-14:07:10,992 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
181004-14:07:11,42 workflow INFO:
	 [Node] Setting-up "grouplevel_affect.grabcbf" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/grouplevel_affect/grabcbf".
181004-14:07:11,52 workflow INFO:
	 [Node] Running "grabcbf" ("nipype.interfaces.io.DataGrabber")
181004-14:07:11,69 workflow INFO:
	 [Node] Finished "grouplevel_affect.grabcbf".
181004-14:07:12,993 workflow INFO:
	 [Job 0] Completed (grouplevel_affect.grabcbf).
181004-14:07:12,997 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free me